In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import svgwrite

# Function to read CSV without headers
def read_csv_no_headers(csv_path):
    """Read CSV file without headers and return structured data."""
    data = np.genfromtxt(csv_path, delimiter=',')
    
    # Assuming the order is [strokes, z, y, x]
    strokes = data[:, 0]
    y = data[:, 2]
    x = data[:, 3]
    
    # Combine x and y into coordinate pairs
    paths = []
    for stroke in np.unique(strokes):
        path = np.column_stack((x[strokes == stroke], y[strokes == stroke]))
        paths.append(path)
    
    return paths

# Function to normalize paths
def normalize_paths(paths):
    """Normalize the paths to a common scale, e.g., between 0 and 1."""
    all_points = np.concatenate(paths, axis=0)
    min_val, max_val = np.min(all_points, axis=0), np.max(all_points, axis=0)
    paths_normalized = [(path - min_val) / (max_val - min_val) for path in paths]
    return paths_normalized, min_val, max_val

# Function to approximate a path with a cubic Bézier curve
def approximate_cubic_bezier(path):
    """Approximate a path with a cubic Bézier curve."""
    n = len(path)
    if n < 4:
        # Pad or repeat points to create a cubic Bézier
        padded_path = np.pad(path, ((0, 4-n), (0, 0)), mode='edge')
        return padded_path
    
    # Simplified approach: take the first 4 points
    bezier_curve = path[:4]
    return bezier_curve

# Function to convert paths to Bézier curves
def paths_to_bezier(paths):
    """Convert paths to Bézier curves."""
    beziers = []
    for path in paths:
        bezier_curve = approximate_cubic_bezier(path)
        beziers.append(bezier_curve)
    return beziers

# Function to pad paths to ensure uniform length
def pad_paths(paths, target_length):
    """Pads each path with zeros to ensure uniform length."""
    padded_paths = []
    for path in paths:
        if len(path) < target_length:
            padding = np.zeros((target_length - len(path), 2))
            padded_path = np.vstack([path, padding])
        else:
            padded_path = path[:target_length]
        padded_paths.append(padded_path)
    return np.array(padded_paths)

# Function to align the number of samples between input and solution paths
def align_sample_numbers(input_paths, solution_paths):
    """Align the number of samples between input and solution paths."""
    input_count = len(input_paths)
    solution_count = len(solution_paths)
    
    if input_count > solution_count:
        solution_paths = np.tile(solution_paths, (input_count // solution_count + 1, 1, 1))[:input_count]
    elif solution_count > input_count:
        input_paths = np.tile(input_paths, (solution_count // input_count + 1, 1, 1))[:solution_count]
    
    return input_paths, solution_paths

# Function to convert Bézier curves into an SVG file
def bezier_to_svg(bezier_curves, filename="output.svg"):
    """Convert Bézier curves into an SVG file."""
    dwg = svgwrite.Drawing(filename, profile='tiny')

    for curve in bezier_curves:
        path_data = f"M {curve[0][0]} {curve[0][1]} "  # Move to the start point
        path_data += f"C {curve[1][0]} {curve[1][1]}, {curve[2][0]} {curve[2][1]}, {curve[3][0]} {curve[3][1]}"  # Cubic Bézier command
        dwg.add(dwg.path(d=path_data, stroke=svgwrite.rgb(0, 0, 0, '%'), fill="none"))

    dwg.save()

# Example usage
input_csv_path = 'frag0.csv'
solution_csv_path = 'frag01_sol.csv'

input_paths = read_csv_no_headers(input_csv_path)
solution_paths = read_csv_no_headers(solution_csv_path)

input_paths_normalized, _, _ = normalize_paths(input_paths)
solution_paths_normalized, _, _ = normalize_paths(solution_paths)

# Convert to Bézier curves
input_beziers = paths_to_bezier(input_paths_normalized)
solution_beziers = paths_to_bezier(solution_paths_normalized)

# Align the number of samples
input_beziers_aligned, solution_beziers_aligned = align_sample_numbers(input_beziers, solution_beziers)

# Pad or trim to ensure uniform length
target_length = max(len(bezier) for bezier in input_beziers_aligned)
input_beziers_padded = pad_paths(input_beziers_aligned, target_length)
solution_beziers_padded = pad_paths(solution_beziers_aligned, target_length)

# Flatten the data for input to the model
X_train = np.array([np.ravel(bezier) for bezier in input_beziers_padded])
Y_train = np.array([np.ravel(bezier) for bezier in solution_beziers_padded])

# Check that the input and output data have the same number of samples
assert X_train.shape[0] == Y_train.shape[0], "Mismatch in number of samples"

# Define the model
def create_bezier_model(input_shape):
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(input_shape[0])  # Output size should match the flattened Bézier parameters
    ])
    return model

input_shape = (X_train.shape[1],)
bezier_model = create_bezier_model(input_shape)

# Compile the model
bezier_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
bezier_model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.1)

# Call the visualization function for both input and solution Bézier curves
bezier_to_svg(input_beziers_padded, filename="ip.svg")
bezier_to_svg(solution_beziers_padded, filename="sp.svg")

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 774ms/step - loss: 0.2504 - val_loss: 0.3452
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1772 - val_loss: 0.3382
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.1277 - val_loss: 0.3296
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0967 - val_loss: 0.3198
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0864 - val_loss: 0.3103
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0925 - val_loss: 0.3039
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0995 - val_loss: 0.3015
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0974 - val_loss: 0.3022
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0890 - val_loss: 0.3045
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0807 - val_loss: 0.3072
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.0756 - val_loss: 0.3094
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0740 - val_l

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import svgwrite

# Function to read CSV without headers
def read_csv_no_headers(csv_path):
    """Read CSV file without headers and return structured data."""
    data = np.genfromtxt(csv_path, delimiter=',')
    
    # Assuming the order is [strokes, z, y, x]
    strokes = data[:, 0]
    y = data[:, 2]
    x = data[:, 3]
    
    # Combine x and y into coordinate pairs
    paths = []
    for stroke in np.unique(strokes):
        path = np.column_stack((x[strokes == stroke], y[strokes == stroke]))
        paths.append(path)
    
    return paths

# Function to normalize paths
def normalize_paths(paths):
    """Normalize the paths to a common scale, e.g., between 0 and 1."""
    all_points = np.concatenate(paths, axis=0)
    min_val, max_val = np.min(all_points, axis=0), np.max(all_points, axis=0)
    paths_normalized = [(path - min_val) / (max_val - min_val) for path in paths]
    return paths_normalized, min_val, max_val

# Function to approximate a path with a cubic Bézier curve
def approximate_cubic_bezier(path):
    """Approximate a path with a cubic Bézier curve."""
    n = len(path)
    if n < 4:
        # Pad or repeat points to create a cubic Bézier
        padded_path = np.pad(path, ((0, 4-n), (0, 0)), mode='edge')
        return padded_path
    
    # Simplified approach: take the first 4 points
    bezier_curve = path[:4]
    return bezier_curve

# Function to convert paths to Bézier curves
def paths_to_bezier(paths):
    """Convert paths to Bézier curves."""
    beziers = []
    for path in paths:
        bezier_curve = approximate_cubic_bezier(path)
        beziers.append(bezier_curve)
    return beziers

# Function to pad paths to ensure uniform length
def pad_paths(paths, target_length):
    """Pads each path with zeros to ensure uniform length."""
    padded_paths = []
    for path in paths:
        if len(path) < target_length:
            padding = np.zeros((target_length - len(path), 2))
            padded_path = np.vstack([path, padding])
        else:
            padded_path = path[:target_length]
        padded_paths.append(padded_path)
    return np.array(padded_paths)

# Function to align the number of samples between input and solution paths
def align_sample_numbers(input_paths, solution_paths):
    """Align the number of samples between input and solution paths."""
    input_count = len(input_paths)
    solution_count = len(solution_paths)
    
    if input_count > solution_count:
        solution_paths = np.tile(solution_paths, (input_count // solution_count + 1, 1, 1))[:input_count]
    elif solution_count > input_count:
        input_paths = np.tile(input_paths, (solution_count // input_count + 1, 1, 1))[:solution_count]
    
    return input_paths, solution_paths

# Function to convert Bézier curves into an SVG file
def bezier_to_svg(bezier_curves, filename="output.svg"):
    """Convert Bézier curves into an SVG file."""
    dwg = svgwrite.Drawing(filename, profile='tiny')

    for curve in bezier_curves:
        if len(curve) < 4:
            continue
        path_data = f"M {curve[0][0]} {curve[0][1]} "  # Move to the start point
        path_data += f"C {curve[1][0]} {curve[1][1]}, {curve[2][0]} {curve[2][1]}, {curve[3][0]} {curve[3][1]}"  # Cubic Bézier command
        dwg.add(dwg.path(d=path_data, stroke=svgwrite.rgb(0, 0, 0, '%'), fill="none"))

    dwg.save()

# Example usage
input_csv_path = 'frag0.csv'
solution_csv_path = 'frag01_sol.csv'

input_paths = read_csv_no_headers(input_csv_path)
solution_paths = read_csv_no_headers(solution_csv_path)

input_paths_normalized, _, _ = normalize_paths(input_paths)
solution_paths_normalized, _, _ = normalize_paths(solution_paths)

# Convert to Bézier curves
input_beziers = paths_to_bezier(input_paths_normalized)
solution_beziers = paths_to_bezier(solution_paths_normalized)

# Align the number of samples
input_beziers_aligned, solution_beziers_aligned = align_sample_numbers(input_beziers, solution_beziers)

# Pad or trim to ensure uniform length
target_length = max(len(bezier) for bezier in input_beziers_aligned)
input_beziers_padded = pad_paths(input_beziers_aligned, target_length)
solution_beziers_padded = pad_paths(solution_beziers_aligned, target_length)

# Flatten the data for input to the model
X_train = np.array([np.ravel(bezier) for bezier in input_beziers_padded])
Y_train = np.array([np.ravel(bezier) for bezier in solution_beziers_padded])

# Check that the input and output data have the same number of samples
assert X_train.shape[0] == Y_train.shape[0], "Mismatch in number of samples"

# Define the model
def create_bezier_model(input_shape):
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(input_shape[0])  # Output size should match the flattened Bézier parameters
    ])
    return model

input_shape = (X_train.shape[1],)
bezier_model = create_bezier_model(input_shape)

# Compile the model
bezier_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
bezier_model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.1)

# Visualize the data
bezier_to_svg(input_beziers_padded, filename="input_paths.svg")
bezier_to_svg(solution_beziers_padded, filename="solution_paths.svg")

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 808ms/step - loss: 0.2276 - val_loss: 0.3442
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.1686 - val_loss: 0.3362
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1260 - val_loss: 0.3268
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0986 - val_loss: 0.3163
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0875 - val_loss: 0.3065
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0908 - val_loss: 0.3001
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0962 - val_loss: 0.2978
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.0946 - val_loss: 0.2985
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0875 - val_loss: 0.3007
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0800 - val_loss: 0.3035
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0750 - val_loss: 0.3060
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0728 - val_l

In [1]:
import numpy as np
import svgwrite

def bezier_to_svg(bezier_curves, filename="output.svg"):
    """Convert Bézier curves into an SVG file."""
    # Check if bezier_curves is empty
    if bezier_curves.size == 0:
        print("No Bézier curves to visualize.")
        return
    
    # Create a new SVG drawing
    dwg = svgwrite.Drawing(filename, profile='tiny')

    # Set default stroke color and width
    stroke_color = 'black'
    stroke_width = 2
    
    for curve in bezier_curves:
        if curve.size == 0 or len(curve) < 4:
            print("Insufficient points for a Bézier curve.")
            continue
        
        # Ensure coordinates are valid
        try:
            path_data = f"M {curve[0][0]} {curve[0][1]} "  # Move to the start point
            path_data += f"C {curve[1][0]} {curve[1][1]}, {curve[2][0]} {curve[2][1]}, {curve[3][0]} {curve[3][1]}"  # Cubic Bézier command
            dwg.add(dwg.path(d=path_data, stroke=stroke_color, fill="none", stroke_width=stroke_width))
        except Exception as e:
            print(f"Error creating path: {e}")
    
    dwg.save()
    print(f"SVG file saved as {filename}")

# Example usage
input_csv_path = 'frag0.csv'
solution_csv_path = 'frag01_sol.csv'

def read_csv_no_headers(csv_path):
    """Read CSV file without headers and return structured data."""
    data = np.genfromtxt(csv_path, delimiter=',')
    
    strokes = data[:, 0]
    y = data[:, 2]
    x = data[:, 3]
    
    paths = []
    for stroke in np.unique(strokes):
        path = np.column_stack((x[strokes == stroke], y[strokes == stroke]))
        paths.append(path)
    
    return paths

def normalize_paths(paths):
    """Normalize the paths to a common scale, e.g., between 0 and 1."""
    all_points = np.concatenate(paths, axis=0)
    min_val, max_val = np.min(all_points, axis=0), np.max(all_points, axis=0)
    paths_normalized = [(path - min_val) / (max_val - min_val) for path in paths]
    return paths_normalized, min_val, max_val

def approximate_cubic_bezier(path):
    """Approximate a path with a cubic Bézier curve."""
    n = len(path)
    if n < 4:
        padded_path = np.pad(path, ((0, 4-n), (0, 0)), mode='edge')
        return padded_path
    
    bezier_curve = path[:4]
    return bezier_curve

def paths_to_bezier(paths):
    """Convert paths to Bézier curves."""
    beziers = []
    for path in paths:
        bezier_curve = approximate_cubic_bezier(path)
        beziers.append(bezier_curve)
    return beziers

def pad_paths(paths, target_length):
    """Pads each path with zeros to ensure uniform length."""
    padded_paths = []
    for path in paths:
        if len(path) < target_length:
            padding = np.zeros((target_length - len(path), 2))
            padded_path = np.vstack([path, padding])
        else:
            padded_path = path[:target_length]
        padded_paths.append(padded_path)
    return np.array(padded_paths)

def align_sample_numbers(input_paths, solution_paths):
    """Align the number of samples between input and solution paths."""
    input_count = len(input_paths)
    solution_count = len(solution_paths)
    
    if input_count > solution_count:
        solution_paths = np.tile(solution_paths, (input_count // solution_count + 1, 1, 1))[:input_count]
    elif solution_count > input_count:
        input_paths = np.tile(input_paths, (solution_count // input_count + 1, 1, 1))[:solution_count]
    
    return input_paths, solution_paths

# Read and process data
input_paths = read_csv_no_headers(input_csv_path)
solution_paths = read_csv_no_headers(solution_csv_path)

input_paths_normalized, _, _ = normalize_paths(input_paths)
solution_paths_normalized, _, _ = normalize_paths(solution_paths)

# Convert to Bézier curves
input_beziers = paths_to_bezier(input_paths_normalized)
solution_beziers = paths_to_bezier(solution_paths_normalized)

# Align the number of samples
input_beziers_aligned, solution_beziers_aligned = align_sample_numbers(input_beziers, solution_beziers)

# Pad or trim to ensure uniform length
target_length = max(len(bezier) for bezier in input_beziers_aligned)
input_beziers_padded = pad_paths(input_beziers_aligned, target_length)
solution_beziers_padded = pad_paths(solution_beziers_aligned, target_length)

# Save Bézier curves as SVG
bezier_to_svg(input_beziers_padded, filename="input_pat.svg")
bezier_to_svg(solution_beziers_padded, filename="solution_pat.svg")

SVG file saved as input_pat.svg
SVG file saved as solution_pat.svg


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import svgwrite
import cairosvg

# Function to read CSV without headers
def read_csv_no_headers(csv_path):
    """Read CSV file without headers and return structured data."""
    data = np.genfromtxt(csv_path, delimiter=',')
    
    # Assuming the order is [strokes, z, y, x]
    strokes = data[:, 0]
    y = data[:, 2]
    x = data[:, 3]
    
    # Combine x and y into coordinate pairs
    paths = []
    for stroke in np.unique(strokes):
        path = np.column_stack((x[strokes == stroke], y[strokes == stroke]))
        paths.append(path)
    
    return paths

# Function to normalize paths
def normalize_paths(paths):
    """Normalize the paths to a common scale, e.g., between 0 and 1."""
    all_points = np.concatenate(paths, axis=0)
    min_val, max_val = np.min(all_points, axis=0), np.max(all_points, axis=0)
    paths_normalized = [(path - min_val) / (max_val - min_val) for path in paths]
    return paths_normalized, min_val, max_val

# Function to approximate a path with a cubic Bézier curve
def approximate_cubic_bezier(path):
    """Approximate a path with a cubic Bézier curve."""
    n = len(path)
    if n < 4:
        # Pad or repeat points to create a cubic Bézier
        padded_path = np.pad(path, ((0, 4-n), (0, 0)), mode='edge')
        return padded_path
    
    # Simplified approach: take the first 4 points
    bezier_curve = path[:4]
    return bezier_curve

# Function to convert paths to Bézier curves
def paths_to_bezier(paths):
    """Convert paths to Bézier curves."""
    beziers = []
    for path in paths:
        bezier_curve = approximate_cubic_bezier(path)
        beziers.append(bezier_curve)
    return beziers

# Function to pad paths to ensure uniform length
def pad_paths(paths, target_length):
    """Pads each path with zeros to ensure uniform length."""
    padded_paths = []
    for path in paths:
        if len(path) < target_length:
            padding = np.zeros((target_length - len(path), 2))
            padded_path = np.vstack([path, padding])
        else:
            padded_path = path[:target_length]
        padded_paths.append(padded_path)
    return np.array(padded_paths)

# Function to align the number of samples between input and solution paths
def align_sample_numbers(input_paths, solution_paths):
    """Align the number of samples between input and solution paths."""
    input_count = len(input_paths)
    solution_count = len(solution_paths)
    
    if input_count > solution_count:
        solution_paths = np.tile(solution_paths, (input_count // solution_count + 1, 1, 1))[:input_count]
    elif solution_count > input_count:
        input_paths = np.tile(input_paths, (solution_count // input_count + 1, 1, 1))[:solution_count]
    
    return input_paths, solution_paths

# Function to convert Bézier curves into an SVG file
def bezier_to_svg(bezier_curves, filename="output.svg"):
    """Convert Bézier curves into an SVG file."""
    dwg = svgwrite.Drawing(filename, profile='tiny')

    for curve in bezier_curves:
        path_data = f"M {curve[0][0]} {curve[0][1]} "  # Move to the start point
        path_data += f"C {curve[1][0]} {curve[1][1]}, {curve[2][0]} {curve[2][1]}, {curve[3][0]} {curve[3][1]}"  # Cubic Bézier command
        dwg.add(dwg.path(d=path_data, stroke=svgwrite.rgb(0, 0, 0, '%'), fill="none"))

    dwg.save()

# Function to rasterize SVG to PNG
def polylines2svg(paths_XYs, svg_path):
    """Rasterize polylines into SVG and convert to PNG."""
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))
    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)
    
    # Create a new SVG drawing
    dwg = svgwrite.Drawing(svg_path, profile='tiny', shape_rendering='crispEdges')
    group = dwg.g()
    
    for i, path in enumerate(paths_XYs):
        path_data = []
        for XY in path:
            path_data.append(("M", (XY[0, 0], XY[0, 1])))
            for j in range(1, len(XY)):
                path_data.append(("L", (XY[j, 0], XY[j, 1])))
            if not np.allclose(XY[0], XY[-1]):
                path_data.append(("Z", None))
        group.add(dwg.path(d=path_data, fill='none', stroke='black', stroke_width=2))
    dwg.add(group)
    dwg.save()
    
    png_path = svg_path.replace('.svg', '.png')
    fact = max(1, 1024 // min(H, W))
    cairosvg.svg2png(url=svg_path, write_to=png_path,
                     parent_width=W, parent_height=H,
                     output_width=fact * W, output_height=fact * H,
                     background_color='white')
    return png_path

# Example usage
input_csv_path = 'frag0.csv'
solution_csv_path = 'frag01_sol.csv'

input_paths = read_csv_no_headers(input_csv_path)
solution_paths = read_csv_no_headers(solution_csv_path)

input_paths_normalized, _, _ = normalize_paths(input_paths)
solution_paths_normalized, _, _ = normalize_paths(solution_paths)

# Convert to Bézier curves
input_beziers = paths_to_bezier(input_paths_normalized)
solution_beziers = paths_to_bezier(solution_paths_normalized)

# Align the number of samples
input_beziers_aligned, solution_beziers_aligned = align_sample_numbers(input_beziers, solution_beziers)

# Pad or trim to ensure uniform length
target_length = max(len(bezier) for bezier in input_beziers_aligned)
input_beziers_padded = pad_paths(input_beziers_aligned, target_length)
solution_beziers_padded = pad_paths(solution_beziers_aligned, target_length)

# Flatten the data for input to the model
X_train = np.array([np.ravel(bezier) for bezier in input_beziers_padded])
Y_train = np.array([np.ravel(bezier) for bezier in solution_beziers_padded])

# Check that the input and output data have the same number of samples
assert X_train.shape[0] == Y_train.shape[0], "Mismatch in number of samples"

# Define the model
def create_bezier_model(input_shape):
    model = tf.keras.Sequential([
        layers.Dense(128, activation='relu', input_shape=input_shape),
        layers.Dense(256, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(input_shape[0])  # Output size should match the flattened Bézier parameters
    ])
    return model

input_shape = (X_train.shape[1],)
bezier_model = create_bezier_model(input_shape)

# Compile the model
bezier_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
bezier_model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.1)

# Visualize and rasterize the results
input_svg_path = "input_beziers.svg"
solution_svg_path = "solution_beziers.svg"

bezier_to_svg(input_beziers_padded, filename=input_svg_path)
bezier_to_svg(solution_beziers_padded, filename=solution_svg_path)

input_png_path = polylines2svg(input_beziers_padded, svg_path=input_svg_path)
solution_png_path = polylines2svg(solution_beziers_padded, svg_path=solution_svg_path)

print(f"Input PNG saved to: {input_png_path}")
print(f"Solution PNG saved to: {solution_png_path}")


C:\Users\Ayush Pathak\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 799ms/step - loss: 0.2450 - val_loss: 0.3450
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.1819 - val_loss: 0.3381
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.1355 - val_loss: 0.3299
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.1033 - val_loss: 0.3205
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0866 - val_loss: 0.3107
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0854 - val_loss: 0.3027
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0918 - val_loss: 0.2982
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0944 - val_loss: 0.2970
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0904 - val_loss: 0.2979
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0831 - val_loss: 0.2999
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0763 - val_loss: 0.3020
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.0721 - val_l

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed